Imports

In [15]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib
import tqdm.auto as tqdm

import os
from PIL import Image
from sklearn.metrics import accuracy_score
import torchvision
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc

import torchvision.models as models

plt.style.use('seaborn')

import DiagnosisFunctions.tools as tools

import albumentations as A
import torchvision.transforms.functional as TF

Read files

In [71]:
ground_truths = pd.read_csv('GT.csv')
predictions = pd.read_csv('pred.csv')

Create roc curves

In [64]:
fpr, tpr, thresholds = roc_curve(ground_truths.iloc[:,0], predictions.iloc[:,0])
auc(fpr, tpr)

0.4891871300474712

Set metrics

In [70]:
basic_chars = ['scale','plaque','pustule','patch','papule','dermatoglyph_disruption','open_comedo']
open_area = ['area_acral_distribution','area_exposed_areas','area_extensor_sites','area_seborrheic_region']
diagnosis = ['diagnosis_acne','diagnosis_actinic_keratosis','diagnosis_psoriasis','diagnosis_seborrheic_dermatitis','diagnosis_viral_warts','diagnosis_vitiligo']

In [61]:
thresholds = np.arange(0, 1, 0.01)
def predict(y_est_prob, threshold):
	return (y_est_prob >= threshold).astype('int')

In [88]:
# Basic characteristics
basic_chars_thresholds = []
basic_chars_scores = []
for char in basic_chars:
    scores = [f1_score(ground_truths[char], predict(predictions[char], t)) for t in thresholds]
    i_opt = np.argmax(scores)
    basic_chars_thresholds.append(thresholds[i_opt])
    basic_chars_scores.append(scores[i_opt])
thresholds_scores_opt = pd.DataFrame(
    [basic_chars_thresholds, basic_chars_scores],
    index=['threshold','score'],
    columns=basic_chars)

In [89]:
thresholds_scores_opt

,scale,plaque,pustule,patch,papule,dermatoglyph_disruption,open_comedo
threshold,0.000000,0.000000,0.11000,0.040000,0.260000,0.170000,0.050000
score,0.727711,0.818792,0.35461,0.651282,0.700767,0.192771,0.376997


In [99]:
thresholds_final = thresholds_scores_opt.loc['threshold',:]
thresholds_final

scale                      0.00
plaque                     0.00
pustule                    0.11
patch                      0.04
papule                     0.26
dermatoglyph_disruption    0.17
open_comedo                0.05
Name: threshold, dtype: float64

In [ ]:
# Find Arg max


In [103]:
predictions_final = predictions
for char in basic_chars:
    predictions_final[char] = predictions_final[char].apply(lambda x: predict(x,thresholds_final[char]))

In [104]:
predictions_final

,scale,plaque,pustule,patch,papule,dermatoglyph_disruption,open_comedo,diagnosis_acne,diagnosis_actinic_keratosis,diagnosis_psoriasis,diagnosis_seborrheic_dermatitis,diagnosis_viral_warts,diagnosis_vitiligo,area_acral_distribution,area_exposed_areas,area_extensor_sites,area_seborrheic_region
0,1,1,1,1,1,0,1,0.137347,0.125301,0.100918,0.189174,0.056564,0.390696,0.182020,0.191463,0.226526,0.399990
1,1,1,1,1,0,0,1,0.656633,0.007493,0.017652,0.012336,0.004294,0.301593,0.273623,0.210103,0.464420,0.051854
2,1,1,0,1,1,0,1,0.119855,0.090334,0.143107,0.170261,0.061037,0.415406,0.263855,0.216426,0.286193,0.233526
3,1,1,1,1,1,0,1,0.133144,0.118501,0.104148,0.184419,0.057094,0.402695,0.187883,0.192948,0.232952,0.386218
4,1,1,1,1,1,0,1,0.143213,0.098048,0.158958,0.206276,0.088736,0.304771,0.246189,0.287395,0.268474,0.197941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,1,1,1,1,1,0,1,0.205466,0.495803,0.057748,0.126564,0.043132,0.071287,0.141697,0.352642,0.122740,0.382921
260,1,1,1,1,1,0,1,0.159024,0.622436,0.032360,0.122365,0.025718,0.038097,0.131362,0.292158,0.081341,0.495139
261,1,1,1,1,1,0,1,0.141435,0.608623,0.039653,0.106519,0.043177,0.060593,0.142031,0.328102,0.104197,0.425671
262,1,1,1,1,1,0,1,0.145566,0.591937,0.047249,0.074877,0.057762,0.082608,0.161017,0.357441,0.148282,0.333260


Tests

In [57]:
gt1 = ground_truths.iloc[:,0]
pred1 = predictions.iloc[:,0]

In [63]:
# General issue with f1
idx = np.argmax(scores)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[idx], scores[idx]))
for i in range(0,len(scores)):
    print(f'Threshold={thresholds[i]:.2f}, f1-Score={scores[i]:.3f}')

Best Threshold=0.000000, F-Score=0.650
Threshold=0.00, f1-Score=0.650
Threshold=0.05, f1-Score=0.649
Threshold=0.10, f1-Score=0.642
Threshold=0.15, f1-Score=0.632
Threshold=0.20, f1-Score=0.629
Threshold=0.25, f1-Score=0.606
Threshold=0.30, f1-Score=0.594
Threshold=0.35, f1-Score=0.616
Threshold=0.40, f1-Score=0.577
Threshold=0.45, f1-Score=0.557
Threshold=0.50, f1-Score=0.540
Threshold=0.55, f1-Score=0.427
Threshold=0.60, f1-Score=0.351
Threshold=0.65, f1-Score=0.287
Threshold=0.70, f1-Score=0.216
Threshold=0.75, f1-Score=0.153
Threshold=0.80, f1-Score=0.083
Threshold=0.85, f1-Score=0.000
Threshold=0.90, f1-Score=0.000
Threshold=0.95, f1-Score=0.000


In [41]:
gt1.sum() / len(gt1)

0.571969696969697